# Notebook overview
Trains a Multi-Layer Perceptron (MLP) using PyTorch on precomputed image embeddings (DINOv2) for fine-grained classification.

- Loads image embeddings and species labels from CSV/PyTorch files
- Implements a custom dataset class for efficient loading of embedding tensors
- Defines an MLP architecture with linear layers, Tanh activation, and label smoothing
- Uses AdamW optimizer with a sequential learning rate strategy (Linear Warmup followed by Cosine Annealing)
- Addresses class imbalance using a WeightedRandomSampler
- Records training process, balanced accuracy metrics, and saves the best model

The notebook was exported as a Python script and run in a console using Tmux to execute it. The notebook was used for both origin and resized dataset just adapte the paths.

# Preperations

### Imports

In [110]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import copy

import torch
import torch.nn as nn
import torch.optim as optim
# from custom_dataset import CustomDataset
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR

import time
from datetime import datetime
from sklearn import metrics
import matplotlib.pyplot as plt

### Variable - SEED

In [111]:
# for reproducibility
SEED = 42

torch.manual_seed(SEED)               # PyTorch CPU
torch.cuda.manual_seed_all(SEED)      # PyTorch GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Load Paths - train_path_high_res, val_path_high_res, label_map_id_path, embedding_dir_path, result_dir_path

In [ ]:
# train id
TRAIN_PATH_HIGH_RES = r'/home/jleick/masterArbeitProjekt/final_release/data/datasets/created/high_id_train.csv'
train_path_high_res = Path(TRAIN_PATH_HIGH_RES)
if not train_path_high_res.exists():
    raise FileNotFoundError(f"File does not exist: {TRAIN_PATH_HIGH_RES}")

# val id
VAL_PATH_HIGH_RES = r'/home/jleick/masterArbeitProjekt/final_release/data/datasets/created/high_id_val.csv'
val_path_high_res = Path(VAL_PATH_HIGH_RES)
if not val_path_high_res.exists():
    raise FileNotFoundError(f"File does not exist: {VAL_PATH_HIGH_RES}")

# label map
LABEL_MAP_ID_PATH = r'/home/jleick/masterArbeitProjekt/final_release/data/datasets/created/label_map_id.csv'
label_map_id_path = Path(LABEL_MAP_ID_PATH)
if not label_map_id_path.exists():
    raise FileNotFoundError(f"File does not exist: {LABEL_MAP_ID_PATH}")

### Folder to load Embeddings
EMBEDDING_DIR_PATH = r'/home/jleick/masterArbeitProjekt/final_release/data/embeddings/copied/high'
embedding_dir_path = Path(EMBEDDING_DIR_PATH)
if not embedding_dir_path.exists():
    raise FileNotFoundError(f"Folder does not exist: {EMBEDDING_DIR_PATH}")

### Folder to save results
RESULT_DIR_PATH = r'/home/jleick/masterArbeitProjekt/final_release/models/mlp/origin/model'
result_dir_path = Path(RESULT_DIR_PATH)
if not result_dir_path.exists():
    raise FileNotFoundError(f"Folder does not exist: {RESULT_DIR_PATH}")

### Load df - train_path_high_res_df, val_path_high_res_df, label_map_id_df

In [ ]:
# train id
train_path_high_res_df = pd.read_csv(train_path_high_res, index_col=False, usecols=['speciesKey', 'image_path'], nrows=10000 )

# val id
val_path_high_res_df = pd.read_csv(val_path_high_res, index_col=False, usecols=['image_path', 'speciesKey'])

# label_map
label_map_id_df = pd.read_csv(label_map_id_path, index_col=0, usecols=['speciesKey', 'species', 'label'])

### merge df - dfs with label_map

In [ ]:
# train id
train_path_high_res_df_label = train_path_high_res_df.merge(label_map_id_df, how='left', on='speciesKey')

# val id
val_path_high_res_df_label = val_path_high_res_df.merge(label_map_id_df, how='left', on='speciesKey')

print("datasets loaded")

### calculate - frequence_spec_exampels_dict (for class balancing)

In [115]:
# frequence of species exampels as dict
frequence_spec_exampels_dict = train_path_high_res_df_label['label'].value_counts().to_dict()

print("frequence_spec_exampels_dict calculated")

frequence_spec_exampels_dict calculated


### Variable - device

In [116]:
# chance selected device to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


# Function

### Function - save_df

In [117]:
def save_df(df:pd.DataFrame, save_path: Path ):
    df.to_csv( save_path, index=False)
    print("df save to: ", save_path)

### Function - CustomDatasetFineGrain

In [118]:
# Define Dataset Class

class CustomDatasetFineGrain(Dataset):

    def __init__(self, df: pd.DataFrame, embedding_dir_path: Path):
        self.embedding_dir_path = embedding_dir_path
        self.df_reduced = df[['image_path', 'label']].copy() # create dataFrame with relevant columns

    def __len__(self):
        return len(self.df_reduced)

    def __getitem__(self, item: int):
        image_file_path, label = self.df_reduced.iloc[item]
        tensor_file_path = Path(image_file_path).with_suffix('.pt')
        absolute_path = self.embedding_dir_path / tensor_file_path
        tensor = torch.load( absolute_path , weights_only=True )
        return tensor.squeeze(), label # squeeze to remove the first dimension

    def get_file_name(self, item: int):
        image_file_path, label = self.df_reduced.iloc[item]
        tensor_file_path = Path(image_file_path).with_suffix('.pt')
        return tensor_file_path, label

### Apply - CustomDatasetFineGrain

In [119]:
# create dataset
train_dataset = CustomDatasetFineGrain(train_path_high_res_df_label, EMBEDDING_DIR_PATH)
val_dataset = CustomDatasetFineGrain(val_path_high_res_df_label, EMBEDDING_DIR_PATH)

print("Datasets created")

Datasets created


# Model Training

### Parameter and Hyperparameter for training

In [ ]:
###########Elements and Hyperparameters#############
model_docu = '''
model = nn.Sequential(
    nn.Linear(384, 192),
    nn.Tanh(),
    nn.Linear(192, 91),
).to(device=device)
'''
embeddings_created_on = "vit_small_patch14_dinov2.lvd142m"
training_data = train_path_high_res

optimizer_name = "AdamW"
learning_rate = 0.001
weight_decay = 1e-5

scheduler_name_1 = 'LinearLR'
start_factor = 0.1
end_factor = 1.0
total_iters = 2
scheduler_name_2 = 'CosineAnnealingLR'
t_max = 28

loss_function_name = "CrossEntropyLoss"
label_smoothing = 0.1

n_epoch = 30 # adamoptimizer
batch_size = 128
validation_funktion = "balanced accuracy"

class_balancing = "WeightedRandomSampler"
class_balancing_value = "replacement True"

validation_function = "validate_balanced_accuracy"

############################################

# define the model (MLP)
model = nn.Sequential(
    nn.Linear(384, 192),
    nn.Tanh(),
    nn.Linear(192, 91),
).to(device=device)

# AdamW Optimizer mit Weight Decay
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=1e-2)

# Loss mit Label Smoothing
loss_fn = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
# loss_fn = nn.CrossEntropyLoss()

# Scheduler 1: Linear Warmup für 2 Epochen
warmup_scheduler = LinearLR(optimizer, start_factor=start_factor, end_factor=end_factor, total_iters=total_iters)

# Scheduler 2: Cosine Decay für die restlichen 28 Epochen
cosine_scheduler = CosineAnnealingLR(optimizer, T_max=t_max)

# Kombinierter Scheduler
scheduler = SequentialLR(optimizer, schedulers=[warmup_scheduler, cosine_scheduler], milestones=[2])


# Class Balancing
sample_weights = [1.0 / frequence_spec_exampels_dict[label] for _, label in train_dataset]
# calculate weights of classes ( 1/1000 low chance to traw) for classes with many exampels
# high weights for classes (1/100 high chances to draw) with less exampels
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
# replacement=True means that an example is put back into the pot after each draw.
# This means that a sample can be drawn several times - especially with very small classes - which helps to see enough examples from minority classes.

# create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler) # sampler=sampler or shuffle=True
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Parameters Loaded")

Parameters Loaded


In [121]:
# validate with accuracy

def validate(model, val_loader, train_loader, device=device):
    model.eval()
    accuracy = []

    with torch.no_grad():
        for name, loader in [("val", val_loader), ("train", train_loader)]:
            correct = 0
            total = 0

            for i, (embeddings, labels) in enumerate(loader, start=1):
                embeddings = embeddings.to(device)
                labels = labels.to(device)
                print(f"validate embeddings on {name}: {i} Batch of labels loaded")

                prediction = model(embeddings)
                output_labels = torch.max(prediction, dim=1)

                correct += (output_labels.indices == labels).sum().item()  #.type(torch.float32).sum().item()
                total += len(output_labels.indices)
            accuracy.append(correct / total)
    return accuracy[0], accuracy[1]

# Quelle: Stevens et al. (2020), S. 214

In [122]:
# validate with balanced accuracy

def balanced_accuracy(labels, predicted_labels):
    accuracy_class = []
    labels_existing = torch.unique(labels)

    for l in labels_existing:
        l_total = 0
        l_correct = 0
        for true_label, pred_label in zip(labels, predicted_labels):
            if l == true_label:
                l_total += 1
                if pred_label == l:
                    l_correct += 1
        
        if l_total > 0:
            l_accuracy = l_correct / l_total
            accuracy_class.append(l_accuracy)

    return sum(accuracy_class) / len(accuracy_class)

def validate_balanced_accuracy(model, val_loader, train_loader, device=device):
    model.eval()
    accuracy = []

    with torch.no_grad():
        for name, loader in [("val", val_loader), ("train", train_loader)]:
            # correct = 0
            # total = 0

            for i, (embeddings, labels) in enumerate(loader, start=1):
                embeddings = embeddings.to(device)
                labels = labels.to(device)
                print(f"validate embeddings on {name}: {i} Batch of labels loaded")

                prediction = model(embeddings)
                output_labels = torch.max(prediction, dim=1)

                accuracy.append( balanced_accuracy(labels, output_labels.indices) )

                # correct += (output_labels.indices == labels).sum().item()  #.type(torch.float32).sum().item()
                # total += len(output_labels.indices)
            # accuracy.append(correct / total)
    return accuracy[0], accuracy[1]

# Balanced accuracy are handelt differently in binary case
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score

In [123]:
def train(model, optimizer, scheduler, loss_fn, n_epoch, train_loader, val_loader, device=device):
    accuracies_val_list = []
    accuracies_train_list = []
    loss_values = []
    lr_list = []
    max_accuracy_val = 0
    best_models = []

    for epoch in range(n_epoch):
        model.train()  # Set model to training mode
        epoch_loss = 0

        lr = optimizer.param_groups[0]['lr']
        lr_list.append(lr)

        for i, (embeddings, labels) in enumerate(train_loader, start=1):
            embeddings = embeddings.to(device)
            labels = labels.to(device)
            print(f"train embeddings: {i} Batch of labels loaded")

            predictions = model(embeddings)
            # print(f"Predictions: {predictions}")
            # print(f"Labels: {labels}")
            loss = loss_fn(predictions, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        loss_values.append(loss.item())
        scheduler.step()  # Update the learning rate
        
        # Validate the model
        # accuracy_val, accuracy_train = validate(model, val_loader, train_loader, device=device)
        accuracy_val, accuracy_train = validate_balanced_accuracy(model, val_loader, train_loader, device=device)
        accuracies_val_list.append(accuracy_val)
        accuracies_train_list.append(accuracy_train)

        # Save the best model
        if accuracy_val > max_accuracy_val:
            best_models.append( copy.deepcopy(model) )
            max_accuracy_val = accuracy_val
            print(f"Saving Best Model with Accuracy: {accuracy_val:.2f}%")

        print(f"Epoch: {epoch+1}, Loss: {epoch_loss:.4f}, Train: {accuracy_train:.2f}, Accuracy: {accuracy_val:.2f}% , Learning Rate: {lr:.6f}, Max Accuracy: {max_accuracy_val:.2f}%")

    return best_models, accuracies_val_list, accuracies_train_list, loss_values, lr_list, max_accuracy_val

# Quelle: Stevens et al. (2020), S. 212-214

### Training - model

In [124]:
start = datetime.now()
best_models, accuracies_val_list, accuracies_train_list, loss_values, lr_list, max_accuracy_val = train(model, optimizer, scheduler, loss_fn, n_epoch, train_loader, val_loader, device=device)

end = datetime.now()

# Calculate the time needed to learn
run_duration = end - start

train embeddings: 1 Batch of labels loaded
train embeddings: 2 Batch of labels loaded
train embeddings: 3 Batch of labels loaded
train embeddings: 4 Batch of labels loaded
train embeddings: 5 Batch of labels loaded
train embeddings: 6 Batch of labels loaded
train embeddings: 7 Batch of labels loaded
train embeddings: 8 Batch of labels loaded
validate embeddings on val: 1 Batch of labels loaded
validate embeddings on val: 2 Batch of labels loaded
validate embeddings on val: 3 Batch of labels loaded
validate embeddings on val: 4 Batch of labels loaded
validate embeddings on val: 5 Batch of labels loaded
validate embeddings on val: 6 Batch of labels loaded
validate embeddings on val: 7 Batch of labels loaded
validate embeddings on val: 8 Batch of labels loaded
validate embeddings on val: 9 Batch of labels loaded
validate embeddings on val: 10 Batch of labels loaded
validate embeddings on val: 11 Batch of labels loaded
validate embeddings on val: 12 Batch of labels loaded
validate embeddin

/home/jleick/miniconda3/envs/masterArbeit/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:198: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


validate embeddings on val: 2 Batch of labels loaded
validate embeddings on val: 3 Batch of labels loaded
validate embeddings on val: 4 Batch of labels loaded
validate embeddings on val: 5 Batch of labels loaded
validate embeddings on val: 6 Batch of labels loaded
validate embeddings on val: 7 Batch of labels loaded
validate embeddings on val: 8 Batch of labels loaded
validate embeddings on val: 9 Batch of labels loaded
validate embeddings on val: 10 Batch of labels loaded
validate embeddings on val: 11 Batch of labels loaded
validate embeddings on val: 12 Batch of labels loaded
validate embeddings on val: 13 Batch of labels loaded
validate embeddings on val: 14 Batch of labels loaded
validate embeddings on val: 15 Batch of labels loaded
validate embeddings on val: 16 Batch of labels loaded
validate embeddings on val: 17 Batch of labels loaded
validate embeddings on val: 18 Batch of labels loaded
validate embeddings on val: 19 Batch of labels loaded
validate embeddings on val: 20 Batch

### Save - model

In [ ]:
# Save best the model
best_model = best_models[-1]

torch.save(best_model.state_dict(), result_dir_path / "model_weights.pt")
# model.load_state_dict(torch.load(PATH_SAVE_MODEL_PARAMETER, weights_only = True), map_location=device) # Model hast to be identically

### Save - learn_process_df

In [ ]:
# Save training process data as df
learn_process_df = pd.DataFrame(
    {
        "Epoch": range(1, n_epoch + 1),
        "Validation_Balanced_Accuracy": accuracies_val_list,
        "Train_Balanced_Accuracy": accuracies_train_list,
        "Loss (Last Batch)": loss_values,
        "Learning_Rate": lr_list,
        "Max_Validation_Balanced_Accuracy": max_accuracy_val,
    }
)

save_df( learn_process_df,  result_dir_path / "learn_process.csv" )

df save to:  /home/jleick/masterArbeitProjekt/final_release/models/mlp/origin/model/_learn_process.csv


In [ ]:
# Results by using testing data

# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Saving Best Model with Accuracy: 0.03%
# Epoch: 1, Loss: 20.5124, Train: 0.00, Accuracy: 0.03% , Learning Rate: 0.000100, Max Accuracy: 0.03%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# /home/jleick/miniconda3/envs/masterArbeit/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
#   warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Saving Best Model with Accuracy: 0.09%
# Epoch: 2, Loss: 15.9993, Train: 0.12, Accuracy: 0.09% , Learning Rate: 0.000550, Max Accuracy: 0.09%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Saving Best Model with Accuracy: 0.14%
# Epoch: 3, Loss: 9.1959, Train: 0.14, Accuracy: 0.14% , Learning Rate: 0.001000, Max Accuracy: 0.14%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 4, Loss: 5.7900, Train: 0.15, Accuracy: 0.14% , Learning Rate: 0.000997, Max Accuracy: 0.14%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Saving Best Model with Accuracy: 0.17%
# Epoch: 5, Loss: 4.3718, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000987, Max Accuracy: 0.17%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Saving Best Model with Accuracy: 0.18%
# Epoch: 6, Loss: 3.9321, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000972, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 7, Loss: 3.7223, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000950, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 8, Loss: 3.6286, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000923, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 9, Loss: 3.6711, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000891, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 10, Loss: 3.6163, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000854, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 11, Loss: 3.5860, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000812, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 12, Loss: 3.5525, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000766, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 13, Loss: 3.4849, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000717, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 14, Loss: 3.4894, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000665, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 15, Loss: 3.4806, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000611, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 16, Loss: 3.4470, Train: 0.17, Accuracy: 0.18% , Learning Rate: 0.000556, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 17, Loss: 3.4073, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000500, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 18, Loss: 3.4884, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000444, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 19, Loss: 3.4172, Train: 0.17, Accuracy: 0.18% , Learning Rate: 0.000389, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 20, Loss: 3.4056, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000335, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 21, Loss: 3.4121, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000283, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 22, Loss: 3.4068, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000234, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 23, Loss: 3.4095, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000188, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 24, Loss: 3.4128, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000146, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 25, Loss: 3.4072, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000109, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 26, Loss: 3.3975, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000077, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 27, Loss: 3.4030, Train: 0.18, Accuracy: 0.17% , Learning Rate: 0.000050, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 28, Loss: 3.4162, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000028, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 29, Loss: 3.3983, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000013, Max Accuracy: 0.18%
# train embeddings: 1 Batch of labels loaded
# train embeddings: 2 Batch of labels loaded
# train embeddings: 3 Batch of labels loaded
# train embeddings: 4 Batch of labels loaded
# validate embeddings on val: 1 Batch of labels loaded
# validate embeddings on val: 2 Batch of labels loaded
# validate embeddings on val: 3 Batch of labels loaded
# validate embeddings on val: 4 Batch of labels loaded
# validate embeddings on train: 1 Batch of labels loaded
# validate embeddings on train: 2 Batch of labels loaded
# validate embeddings on train: 3 Batch of labels loaded
# validate embeddings on train: 4 Batch of labels loaded
# Epoch: 30, Loss: 3.3895, Train: 0.18, Accuracy: 0.18% , Learning Rate: 0.000003, Max Accuracy: 0.18%

In [ ]:
# Saving Best Model with Accuracy: 0.01%
# Epoch: 1, Loss: 20.3221, Train: 0.05, Accuracy: 0.01% , Learning Rate: 0.000100, Max Accuracy: 0.01%
# /home/jleick/miniconda3/envs/masterArbeit/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
#   warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
# Saving Best Model with Accuracy: 0.10%
# Epoch: 2, Loss: 16.0065, Train: 0.60, Accuracy: 0.10% , Learning Rate: 0.000550, Max Accuracy: 0.10%
# Saving Best Model with Accuracy: 0.15%
# Epoch: 3, Loss: 8.8682, Train: 0.83, Accuracy: 0.15% , Learning Rate: 0.001000, Max Accuracy: 0.15%
# Saving Best Model with Accuracy: 0.17%
# Epoch: 4, Loss: 5.5239, Train: 0.97, Accuracy: 0.17% , Learning Rate: 0.000997, Max Accuracy: 0.17%
# Saving Best Model with Accuracy: 0.19%
# Epoch: 5, Loss: 4.3360, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000987, Max Accuracy: 0.19%
# Saving Best Model with Accuracy: 0.19%
# Epoch: 6, Loss: 3.8589, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000972, Max Accuracy: 0.19%
# Epoch: 7, Loss: 3.7316, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000950, Max Accuracy: 0.19%
# Epoch: 8, Loss: 3.6846, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000923, Max Accuracy: 0.19%
# Epoch: 9, Loss: 3.6477, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000891, Max Accuracy: 0.19%
# Epoch: 10, Loss: 3.6082, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000854, Max Accuracy: 0.19%
# Epoch: 11, Loss: 3.5651, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000812, Max Accuracy: 0.19%
# Epoch: 12, Loss: 3.5278, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000766, Max Accuracy: 0.19%
# Epoch: 13, Loss: 3.4970, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000717, Max Accuracy: 0.19%
# Epoch: 14, Loss: 3.4742, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000665, Max Accuracy: 0.19%
# Epoch: 15, Loss: 3.4555, Train: 0.99, Accuracy: 0.19% , Learning Rate: 0.000611, Max Accuracy: 0.19%
# Epoch: 16, Loss: 3.4423, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000556, Max Accuracy: 0.19%
# Epoch: 17, Loss: 3.4290, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000500, Max Accuracy: 0.19%
# Epoch: 18, Loss: 3.4201, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000444, Max Accuracy: 0.19%
# Epoch: 19, Loss: 3.4122, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000389, Max Accuracy: 0.19%
# Epoch: 20, Loss: 3.4062, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000335, Max Accuracy: 0.19%
# Epoch: 21, Loss: 3.3997, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000283, Max Accuracy: 0.19%
# Epoch: 22, Loss: 3.3969, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000234, Max Accuracy: 0.19%
# Epoch: 23, Loss: 3.3934, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000188, Max Accuracy: 0.19%
# Epoch: 24, Loss: 3.3896, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000146, Max Accuracy: 0.19%
# Epoch: 25, Loss: 3.3889, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000109, Max Accuracy: 0.19%
# Epoch: 26, Loss: 3.3872, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000077, Max Accuracy: 0.19%
# Epoch: 27, Loss: 3.3879, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000050, Max Accuracy: 0.19%
# Epoch: 28, Loss: 3.3866, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000028, Max Accuracy: 0.19%
# Epoch: 29, Loss: 3.3856, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000013, Max Accuracy: 0.19%
# Epoch: 30, Loss: 3.3853, Train: 1.00, Accuracy: 0.19% , Learning Rate: 0.000003, Max Accuracy: 0.19%

### Save - train_parameter_df

In [ ]:
train_parameter_df = pd.DataFrame([
    # General
    {"Category": "General", "Name": "Seed", "Value": SEED},
    {"Category": "General", "Name": "start_time", "Value": start},
    {"Category": "General", "Name": "end_time", "Value": end},
    {"Category": "General", "Name": "duration", "Value": run_duration},
    {"Category": "General", "Name": "embeddings_created_on", "Value": embeddings_created_on},
    {"Category": "General", "Name": "data_split", "Value": training_data},
    {"Category": "General", "Name": "validation function", "Value": validate_balanced_accuracy},

    # Model
    {"Category": "Model", "Name": "Architecture", "Value": model_docu},

    # Optimizer
    {"Category": "Optimizer", "Name": "Name", "Value": optimizer_name},
    {"Category": "Optimizer", "Name": "learning_rate", "Value": learning_rate},
    {"Category": "Optimizer", "Name": "weight_decay", "Value": weight_decay},

    # Scheduler
    {"Category": "Scheduler", "Name": f"{scheduler_name_1}_start_factor", "Value": start_factor},
    {"Category": "Scheduler", "Name": f"{scheduler_name_1}_end_factor", "Value": end_factor},
    {"Category": "Scheduler", "Name": f"{scheduler_name_2}_t_max", "Value": t_max},

    # Training Parameters
    {"Category": "Parameter", "Name": "n_epoch", "Value": n_epoch},
    {"Category": "Parameter", "Name": "batch_size", "Value": batch_size},

    # class Balancing
    {"Category": "class_Balancing", "Name": class_balancing, "Value": class_balancing_value},

    # Loss Function
    {"Category": "Loss", "Name": "loss_function", "Value": loss_function_name},
    {"Category": "Loss", "Name": "label_smoothing", "Value": label_smoothing},
])

save_df( train_parameter_df, result_dir_path / "train_parameter.csv" )

df save to:  /home/jleick/masterArbeitProjekt/final_release/models/mlp/origin/model/_train_parameter.csv


# Train Docu